In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv

In [2]:
dataset = Planetoid(root='./', name = 'cora', transform = T.NormalizeFeatures())
g = dataset[0]

In [3]:
print(dataset.num_classes)
print(g)
print(g.keys)

7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
['x', 'train_mask', 'test_mask', 'val_mask', 'y', 'edge_index']


In [4]:
# class GAT(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(GAT, self).__init__()
        
#         self.conv1 = GATConv(in_channels, 8, heads = 8, dropout = 0.6)
#         self.conv2 = GATConv(8 * 8, out_channels, heads = 1, concat = False, dropout = 0.6)
        
#     def forward(self, x, edge_index):
#         x = F.elu(self.conv1(x, edge_index))
#         x = F.dropout(x, p=0.6, training=self.training)
#         x = self.conv2(x, edge_index)
#         return x

In [5]:
class GAT(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GAT, self).__init__()
        
        self.conv1 = GATConv(in_channels, 8, heads = 8, dropout = 0.6)
        self.conv2 = GATConv(8 * 8, out_channels, heads = 1, concat = False, dropout = 0.6)
        
    def forward(self, g):
        x, edge_index = g.x, g.edge_index
            
        x = F.elu(self.conv1(x, edge_index))            
        x = F.dropout(x, p = 0.6, training = self.training)
        x = self.conv2(x, edge_index)
        return x

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
model = GAT(g.num_features, dataset.num_classes).to(device)
g = g.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005, weight_decay = 5e-4)
criterion = nn.CrossEntropyLoss()
print(model)

GAT(
  (conv1): GATConv(1433, 8, heads=8)
  (conv2): GATConv(64, 7, heads=1)
)


In [8]:
def train(data):
    model.train()
    
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    pred = out.argmax(1)
    correct = (pred[data.train_mask] == data.y[data.train_mask]).sum().item()
    train_acc = correct / (data.train_mask.sum().item())
    
    if i % 10 == 0:
        print("Epoch {:05d}  /  Loss {:.4f}  /  Train_acc {:.4f}".
              format(i, loss.item(), train_acc))

In [9]:
def test(data):
    model.eval()
    
    out = model(data)
    pred = out.argmax(1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum().item()
    test_acc = correct / data.test_mask.sum().item()
    print("Test Accuracy :", test_acc)

In [10]:
epoch = 100
for i in range(epoch):
    train(g)
    
test(g)

Epoch 00000  /  Loss 1.9466  /  Train_acc 0.1429
Epoch 00010  /  Loss 1.8522  /  Train_acc 0.6643
Epoch 00020  /  Loss 1.7411  /  Train_acc 0.7429
Epoch 00030  /  Loss 1.5837  /  Train_acc 0.7857
Epoch 00040  /  Loss 1.3688  /  Train_acc 0.8000
Epoch 00050  /  Loss 1.2331  /  Train_acc 0.8143
Epoch 00060  /  Loss 1.0795  /  Train_acc 0.8500
Epoch 00070  /  Loss 0.9842  /  Train_acc 0.8786
Epoch 00080  /  Loss 0.9058  /  Train_acc 0.8714
Epoch 00090  /  Loss 0.7630  /  Train_acc 0.8643
Test Accuracy : 0.814
